In [12]:
import pandas as pd

In [13]:
numeric_columns = [
 'requester_account_age_in_days_at_request',
 'requester_days_since_first_post_on_raop_at_request',
 'requester_number_of_comments_at_request',
 'requester_number_of_comments_in_raop_at_request',
 'requester_number_of_posts_at_request',
 'requester_number_of_posts_on_raop_at_request',
 'requester_number_of_subreddits_at_request',
 'requester_upvotes_minus_downvotes_at_request',
 'requester_upvotes_plus_downvotes_at_request',
 'unix_timestamp_of_request',
 'unix_timestamp_of_request_utc',
 'requester_received_pizza']

def read_data(mode):
    # WARNING: THIS FUNCTION IS DROPPING SPARSE COLUMNS
#     dataframe = pd.DataFrame()
#     dataframe.columns=numeric_columns
    if mode == 'train':
        dataframe = pd.read_json('./data/train.json')
        dataframe['requester_received_pizza'] = dataframe['requester_received_pizza'].astype(int)
    elif mode == 'test':
        dataframe = pd.read_json('./data/test.json')
        
    return dataframe
    

# def x_y_split(dataframe):
#     """ Splits a pd dataframe into the features and results """
#     features = dataframe.drop('requester_received_pizza', axis=1)
#     results = dataframe['requester_received_pizza']
#     return features, results
    

In [14]:
train_df = read_data('train')
train_df[numeric_columns].to_csv('./data/train.csv', index=False, header=None)
# test_df = read_data('test')
# test_df[numeric_columns[:-1]].to_csv('./data/test.csv', index=False)

In [15]:
# test_df[numeric_columns].to_csv('./data/train.csv')
#list(train_df.columns.intersection(test_df.columns))

In [16]:
#list(pd.read_csv('./data/train.csv').columns)

In [17]:
import tensorflow as tf

In [18]:
record_defaults = [tf.constant([], dtype=tf.float32) for c in range(len(numeric_columns))]
record_defaults

[<tf.Tensor 'Const_12:0' shape=(0,) dtype=float32>,
 <tf.Tensor 'Const_13:0' shape=(0,) dtype=float32>,
 <tf.Tensor 'Const_14:0' shape=(0,) dtype=float32>,
 <tf.Tensor 'Const_15:0' shape=(0,) dtype=float32>,
 <tf.Tensor 'Const_16:0' shape=(0,) dtype=float32>,
 <tf.Tensor 'Const_17:0' shape=(0,) dtype=float32>,
 <tf.Tensor 'Const_18:0' shape=(0,) dtype=float32>,
 <tf.Tensor 'Const_19:0' shape=(0,) dtype=float32>,
 <tf.Tensor 'Const_20:0' shape=(0,) dtype=float32>,
 <tf.Tensor 'Const_21:0' shape=(0,) dtype=float32>,
 <tf.Tensor 'Const_22:0' shape=(0,) dtype=float32>,
 <tf.Tensor 'Const_23:0' shape=(0,) dtype=float32>]

In [19]:
def input_fn(data_file, num_epochs, shuffle, batch_size):
    """ Generate an input function for the Estimator """
    assert tf.gfile.Exists(data_file), (f'{data_file} not found. Please make sure the data file is in the correct location.')
    
    def parse_csv(value):
        print('Parsing', data_file)
        columns = tf.decode_csv(value, record_defaults=record_defaults)
        features = dict(zip(numeric_columns, columns))
        labels = features.pop('requester_received_pizza')
        return features, labels
    
    dataset = tf.contrib.data.TextLineDataset(data_file) # in TF 1.4 data got moved to tf.contrib: https://github.com/tensorflow/tensorflow/releases/tag/v1.4.0
    
    if shuffle:
        dataset = dataset.shuffle(buffer_size=10)
        
    dataset = dataset.map(parse_csv, num_threads=5)
    
    dataset = dataset.repeat(num_epochs)
    dataset = dataset.batch(batch_size)
    
    iterator = dataset.make_one_shot_iterator()
    features, labels = iterator.get_next()
    return features, labels


In [20]:
# categorical columns
# post_was_edited true or false
# requester_received_pizza true or fasle
# requester_user_flair None, "schroom", "PIF"
# everything else numerical

tf_feature_columns = [tf.feature_column.numeric_column(x) for x in numeric_columns[:-1]]
# tensorflow features cannot accept strings. must transform strings to categorical or numerical

In [21]:
model_dir = './models/'
model = tf.estimator.LinearClassifier(
    model_dir=model_dir, 
    feature_columns=tf_feature_columns
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './models/', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


In [22]:
train_data = './data/train.csv'
num_epochs = 1
shuffle = True
batch_size = 30
model.train(input_fn=lambda: input_fn(train_data, num_epochs, True, batch_size))

Parsing ./data/train.csv
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./models/model.ckpt-135
INFO:tensorflow:Saving checkpoints for 136 into ./models/model.ckpt.
INFO:tensorflow:loss = 3.04412e+08, step = 136
INFO:tensorflow:global_step/sec: 617.94
INFO:tensorflow:loss = 3.00445e+08, step = 236 (0.162 sec)
INFO:tensorflow:Saving checkpoints for 270 into ./models/model.ckpt.
INFO:tensorflow:Loss for final step: 6.85664e+07.


In [23]:
model.predict()

Parsing ./data/test.csv
INFO:tensorflow:Starting evaluation at 2018-02-03-21:08:22
INFO:tensorflow:Restoring parameters from ./models/model.ckpt-270


InvalidArgumentError: Expect 12 fields but have 11 in record 0
	 [[Node: DecodeCSV = DecodeCSV[OUT_TYPE=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], field_delim=",", use_quote_delim=true](arg0, DecodeCSV/Placeholder, DecodeCSV/Placeholder_1, DecodeCSV/Placeholder_2, DecodeCSV/Placeholder_3, DecodeCSV/Placeholder_4, DecodeCSV/Placeholder_5, DecodeCSV/Placeholder_6, DecodeCSV/Placeholder_7, DecodeCSV/Placeholder_8, DecodeCSV/Placeholder_9, DecodeCSV/Placeholder_10, DecodeCSV/Placeholder_11)]]
	 [[Node: IteratorGetNext = IteratorGetNext[output_shapes=[[?], [?], [?], [?], [?], [?], [?], [?], [?], [?], [?], [?]], output_types=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](OneShotIterator)]]